# Demo de Simulación de Balance - Cantidad de Inversiones

## Configuración inicial de librerías

In [1]:
# Contiene las clases Operacion, Caja y Banco.
import simula_balance as sb

# Contiene las reglas de renovación, por ejemplo,
# toda la caja a overnight.
import reglas_renovacion as rn

# Contiene las métricas, por ejemplo, el C46.
import metricas as met

# Se usa pandas sólo para desplegar claramente el gap de liquidez.
import pandas as pd

## t = 0: Configuración inicial del Banco

In [3]:
# Monto de capital
capital = 10.0

# Sobre este objeto caen los vencimientos de activos y pasivos.
# De aquí se obtiene el dinero para captaciones y colocaciones,
# toda operación, al iniciarse, tiene su correspondiente movimiento
# en la caja.
caja = sb.Caja()

# Construimos un Banco muy sencillo con una colocación bullet
# y una captación (también bullet).
# Lo primero son las tasas de mercado
mercado = {'on_col': 0.0, 'on_cap':0.0,
           'depo2':0.0312, 'depo3':0.033, 'depo4':0.0348,
           'inversion':0.027, 'colocacion':0.045}

# Variable que establece los plazos y porcentajes de renovación a cada plazo.
# Si tenemos x:y significa que al plazo x (medido en número de períodos)
# se hace un y% del monto necesario.
plazos = {4:100}


# Se construye el crédito
credito = sb.Operacion(mercado['colocacion'], 0, 12, 100, {'id':'1', 'ap':'activo', 'producto':'credito'})

# En esta lista se almacenarán las operaciones que insertaremos inicialmente en el banco.
operaciones = [credito]

# Se construye la mezcla de depósitos
i = 0
for plazo in plazos:
    i += 1
    codigo_tasa = 'depo' + str(plazo)
    ponderador = plazos[plazo] / 100.0
    operaciones.append(sb.Operacion(mercado[codigo_tasa], 0, plazo, ponderador * 90,
                                    {'id':str(i), 'ap':'pasivo', 'producto':'depo'}))
    
# Se da de alta el banco inicial
banco = sb.Banco(caja, operaciones, capital)

Vamos a definir algunas funciones auxiliares que harán menos verboso el ejercicio:

In [4]:
def muestra_gap():
    # se determina la situación de liquidez y
    gap = met.construye_gap_liquidez(banco)
    pd_gap = pd.DataFrame([gap()])

    # se muestra el resultado.
    print "Gap de Liquidez a tiempo t = ", banco.tiempo_actual()
    print pd_gap
    print 'c46: ', met.c46(banco, True)
    return

def muestra_resultados():
    # Calcula y muestra el estado de resultados
    er = banco.estado_resultados()
    resultado = 0.0
    for key in er:
        print key, er[key]
        if key == 'activo':
            for interes in er[key]:
                resultado += er[key][interes]
        else:
            for interes in er[key]:
                resultado -= er[key][interes]
    print "resultado: ", resultado
    return

def muestra_balance():
    # Calcula y muestra el balance
    bal = banco.balance()
    for key in bal:
        print key, bal[key]
    return

def muestra_operaciones(t):
    # Muestra los flujos de las operaciones del banco
    activos = banco.activos()
    print 'Activos:'
    print 'prod\t', 'plazo\t', 'monto\t', 'int_dev\t', 'flujo\t'
    for act in activos:
        print act.atributos()['producto'], act.vencimiento(), act.vencimiento() - t, act.monto(), act.interes_devengado(t), act.flujo()
        
    pasivos = banco.pasivos()
    print 'Pasivos:'
    print 'prod\t', 'plazo\t', 'monto\t', 'int_dev\t', 'flujo\t'
    for pas in pasivos:
        print pas.atributos()['producto'], pas.vencimiento(), pas.vencimiento() - t, pas.monto(), pas.interes_devengado(t), pas.flujo()

def avanza():
    # Se avanza un período en el tiempo,
    banco.avanza()
    return
    
def renueva_depo():
    # Se renueva el depósito
    rn.renueva_depos(banco, plazos)

    # se determina la nueva situación de liquidez y
    # se muestra
    muestra_gap()
    return

def toma_inversiones():
    # Se renueva la inversión
    rn.inversion(banco)

    # se determina la nueva situación de liquidez y
    # se muestra.
    muestra_gap()
    return

def arregla_c46():
    # Se arregla el C46,
    rn.cumple_c46(banco, plazos, True)

    # Se muestra gap
    muestra_gap()
    return

def muestra_cartera():
    # Muestra la cartera de operaciones a fecha actual del banco
    for op in banco.cartera():
        print op
    return

def muestra_ratio():
    # Muestra el ratio inversiones / colocaciones
    print round(banco.inv_sobre_col() * 100)

Se construye y muestra el gap de liquidez inicial del banco.

In [5]:
muestra_gap()

Gap de Liquidez a tiempo t =  0
    0   1   2   3       4   5   6   7   8   9   10  11     12  13  14  15  16
0  0.0   0   0   0 -93.132   0   0   0   0   0   0   0  113.5   0   0   0   0
c46:  -0.0


Se muestra el estado de resultados inicial.

In [6]:
muestra_resultados()

activo {'intereses_devengados': 0.0, 'intereses_recibidos': 0.0}
pasivo {'intereses_pagados': 0.0, 'intereses_devengados': 0.0}
resultado:  0.0


Se muestra el balance inicial.

In [7]:
muestra_balance()

patrimonio {'capital_pagado': -10.0}
activo {'caja': 0.0, 'credito': 100.0}
pasivo {'depo': -90.0, 'sobregiro': 0.0}


## t = 1: Se avanza un período en el tiempo.

In [8]:
avanza()

Veamos las operaciones y su resultado, para cada operación se muestra:
- activo/pasivo
- producto
- id
- tiempo inicio
- plazo
- monto inicial
- tasa
- interés devengado
- interés recibido/pagado

In [9]:
muestra_cartera()

('activo', 'credito', '1', 0.0, 12.0, 100.0, 0.045, 1.125, 0.0)
('pasivo', 'depo', '1', 0.0, 4.0, -90.0, 0.0348, -0.783, -0.0)


Verifiquemos el cálculo de interés devengado:

In [10]:
print 'credito:', 100 * 0.045 * 3 / 12.0
print 'depo:', -90 * .0348 * 3 / 12.0 

credito: 1.125
depo: -0.783


Veamos el gap de liquidez y C46.

In [11]:
muestra_gap()

Gap de Liquidez a tiempo t =  1
    0   1   2       3   4   5   6   7   8   9   10     11  12  13  14  15  16
0  0.0   0   0 -93.132   0   0   0   0   0   0   0  113.5   0   0   0   0   0
c46:  -0.0


## t = 2: Se avanza nuevamente en el tiempo.

In [12]:
avanza()

Veamos el resultado de las operaciones:

In [13]:
muestra_cartera()

('activo', 'credito', '1', 0.0, 12.0, 100.0, 0.045, 2.25, 0.0)
('pasivo', 'depo', '1', 0.0, 4.0, -90.0, 0.0348, -1.566, -0.0)


Chequeo de intereses:

In [15]:
print 'credito:', round(100 * 0.045 * 6 / 12, 4)
print 'depo-1:', round(-90 * .0348 * 6 / 12, 4)

credito: 2.25
depo-1: -1.566


Veamos el gap de liquidez.

In [16]:
muestra_gap()

Gap de Liquidez a tiempo t =  2
    0   1       2   3   4   5   6   7   8   9      10  11  12  13  14  15  16
0  0.0   0 -93.132   0   0   0   0   0   0   0  113.5   0   0   0   0   0   0
c46:  -0.0


## t = 3: Se avanza nuevamente en el tiempo.

In [17]:
avanza()

Veamos resultado.

In [18]:
muestra_cartera()

('activo', 'credito', '1', 0.0, 12.0, 100.0, 0.045, 3.375, 0.0)
('pasivo', 'depo', '1', 0.0, 4.0, -90.0, 0.0348, -2.349, -0.0)


Verifiquemos:

In [19]:
print 'credito:', round(100 * 0.045 * 9 / 12, 4)
print 'depo-1:', round(-90 * .0348 * 9 / 12, 4)

credito: 3.375
depo-1: -2.349


Veamos el gap de liquidez.

In [20]:
muestra_gap()

Gap de Liquidez a tiempo t =  3
    0       1   2   3   4   5   6   7   8      9   10  11  12  13  14  15  16
0  0.0 -93.132   0   0   0   0   0   0   0  113.5   0   0   0   0   0   0   0
c46:  93.132


Se arregla el C46.

In [21]:
arregla_c46()

Gap de Liquidez a tiempo t =  3
    0       1   2   3       4   5   6   7   8      9   10  11  12  13  14  15  \
0  0.0  0.6286   0   0 -96.373   0   0   0   0  113.5   0   0   0   0   0   0   

   16  
0   0  
c46:  -0.6286


In [22]:
muestra_cartera()

('activo', 'credito', '1', 0.0, 12.0, 100.0, 0.045, 3.375, 0.0)
('activo', 'inversion', '3-inv', 3.0, 4.0, 93.132, 0.027, 0.0, 0.0)
('pasivo', 'depo', '1', 0.0, 4.0, -90.0, 0.0348, -2.349, -0.0)
('pasivo', 'depo', '3-fin_inv', 3.0, 7.0, -93.132, 0.0348, -0.0, -0.0)


## t = 4. Volvemos a avanzar.

In [23]:
avanza()

Veamos resultados.

In [24]:
muestra_cartera()

('activo', 'credito', '1', 0.0, 12.0, 100.0, 0.045, 4.5, 0.0)
('activo', 'inversion', '3-inv', 3.0, 4.0, 93.132, 0.027, 0.0, 0.6286)
('pasivo', 'depo', '1', 0.0, 4.0, -90.0, 0.0348, -0.0, -3.132)
('pasivo', 'depo', '3-fin_inv', 3.0, 7.0, -93.132, 0.0348, -0.8102, -0.0)


Verifiquemos.

In [32]:
print 'credito:', round(100 * 0.045 * 12 / 12, 4)
print '3-inv:', round(93.132 * 0.027 * 3 / 12, 4)
print 'depo-1:', round(-90 * .0348 * 12 / 12, 4)
print '3-fin_inv', round(-93.132 * 0.0348 * 3 / 12, 4)

credito: 4.5
3-inv: 0.6286
depo-1: -3.132
3-fin_inv -0.8102


Veamos el gap.

In [33]:
muestra_gap()

Gap de Liquidez a tiempo t =  4
       0   1   2       3   4   5   6   7      8   9   10  11  12  13  14  15  \
0  0.6286   0   0 -96.373   0   0   0   0  113.5   0   0   0   0   0   0   0   

   16  
0   0  
c46:  -0.6286


Activo la caja residual.

In [34]:
toma_inversiones()

Gap de Liquidez a tiempo t =  4
    0       1   2       3   4   5   6   7      8   9   10  11  12  13  14  15  \
0 -0.0  0.6328   0 -96.373   0   0   0   0  113.5   0   0   0   0   0   0   0   

   16  
0   0  
c46:  -0.6328


Vemos la cartera.

In [35]:
muestra_cartera()

('activo', 'credito', '1', 0.0, 12.0, 100.0, 0.045, 4.5, 0.0)
('activo', 'inversion', '3-inv', 3.0, 4.0, 93.132, 0.027, 0.0, 0.6286)
('activo', 'inversion', 'inv-4-1', 4.0, 5.0, 0.6286, 0.027, 0.0, 0.0)
('pasivo', 'depo', '1', 0.0, 4.0, -90.0, 0.0348, -0.0, -3.132)
('pasivo', 'depo', '3-fin_inv', 3.0, 7.0, -93.132, 0.0348, -0.8102, -0.0)


## t = 5: Volvemos a avanzar, mostramos el gap y activamos caja.

In [36]:
avanza()

In [37]:
muestra_gap()

Gap de Liquidez a tiempo t =  5
       0   1       2   3   4   5   6      7   8   9   10  11  12  13  14  15  \
0  0.6328   0 -96.373   0   0   0   0  113.5   0   0   0   0   0   0   0   0   

   16  
0   0  
c46:  -0.6328


In [38]:
toma_inversiones()

Gap de Liquidez a tiempo t =  5
    0       1       2   3   4   5   6      7   8   9   10  11  12  13  14  15  \
0 -0.0  0.6371 -96.373   0   0   0   0  113.5   0   0   0   0   0   0   0   0   

   16  
0   0  
c46:  -0.6371


In [39]:
muestra_cartera()

('activo', 'credito', '1', 0.0, 12.0, 100.0, 0.045, 5.625, 0.0)
('activo', 'inversion', '3-inv', 3.0, 4.0, 93.132, 0.027, 0.0, 0.6286)
('activo', 'inversion', 'inv-4-1', 4.0, 5.0, 0.6286, 0.027, 0.0, 0.0042)
('activo', 'inversion', 'inv-5-1', 5.0, 6.0, 0.6328, 0.027, 0.0, 0.0)
('pasivo', 'depo', '1', 0.0, 4.0, -90.0, 0.0348, -0.0, -3.132)
('pasivo', 'depo', '3-fin_inv', 3.0, 7.0, -93.132, 0.0348, -1.6205, -0.0)


## t = 6:Volvemos a avanzar, mostramos el gap y arreglamos C46.

In [40]:
avanza()

In [41]:
muestra_gap()

Gap de Liquidez a tiempo t =  6
       0       1   2   3   4   5      6   7   8   9   10  11  12  13  14  15  \
0  0.6371 -96.373   0   0   0   0  113.5   0   0   0   0   0   0   0   0   0   

   16  
0   0  
c46:  95.7359


In [42]:
arregla_c46()

Gap de Liquidez a tiempo t =  6
    0       1   2   3        4   5      6   7   8   9   10  11  12  13  14  \
0 -0.0  0.0091   0   0 -98.4082   0  113.5   0   0   0   0   0   0   0   0   

   15  16  
0   0   0  
c46:  -0.00909999999999


## t = 7: Volvemos a avanzar, mostramos el gap y arreglamos C46.

In [43]:
avanza()

In [44]:
muestra_gap()

Gap de Liquidez a tiempo t =  7
       0   1   2        3   4      5   6   7   8   9   10  11  12  13  14  15  \
0  0.0091   0   0 -98.4082   0  113.5   0   0   0   0   0   0   0   0   0   0   

   16  
0   0  
c46:  -0.0091


In [45]:
toma_inversiones()

Gap de Liquidez a tiempo t =  7
    0       1   2        3   4      5   6   7   8   9   10  11  12  13  14  \
0 -0.0  0.0092   0 -98.4082   0  113.5   0   0   0   0   0   0   0   0   0   

   15  16  
0   0   0  
c46:  -0.0092


## t = 8. Última iteración.

In [46]:
avanza()

In [47]:
muestra_gap()

Gap de Liquidez a tiempo t =  8
       0   1        2   3      4   5   6   7   8   9   10  11  12  13  14  15  \
0  0.0092   0 -98.4082   0  113.5   0   0   0   0   0   0   0   0   0   0   0   

   16  
0   0  
c46:  -0.0092


In [48]:
toma_inversiones()

Gap de Liquidez a tiempo t =  8
    0       1        2   3      4   5   6   7   8   9   10  11  12  13  14  \
0 -0.0  0.0093 -98.4082   0  113.5   0   0   0   0   0   0   0   0   0   0   

   15  16  
0   0   0  
c46:  -0.0093


In [49]:
muestra_resultados()

activo {'intereses_devengados': 9.0, 'intereses_recibidos': 1.2834}
pasivo {'intereses_pagados': 6.373, 'intereses_devengados': 1.6547}
resultado:  2.2557


In [50]:
muestra_cartera()

('activo', 'credito', '1', 0.0, 12.0, 100.0, 0.045, 9.0, 0.0)
('activo', 'inversion', '3-inv', 3.0, 4.0, 93.132, 0.027, 0.0, 0.6286)
('activo', 'inversion', 'inv-4-1', 4.0, 5.0, 0.6286, 0.027, 0.0, 0.0042)
('activo', 'inversion', 'inv-5-1', 5.0, 6.0, 0.6328, 0.027, 0.0, 0.0043)
('activo', 'inversion', '6-inv', 6.0, 7.0, 95.7359, 0.027, 0.0, 0.6462)
('activo', 'inversion', 'inv-7-1', 7.0, 8.0, 0.0091, 0.027, 0.0, 0.0001)
('activo', 'inversion', 'inv-8-1', 8.0, 9.0, 0.0092, 0.027, 0.0, 0.0)
('pasivo', 'depo', '1', 0.0, 4.0, -90.0, 0.0348, -0.0, -3.132)
('pasivo', 'depo', '3-fin_inv', 3.0, 7.0, -93.132, 0.0348, -0.0, -3.241)
('pasivo', 'depo', '6-fin_inv', 6.0, 10.0, -95.0988, 0.0348, -1.6547, -0.0)
